In [7]:
import numpy as np
glove_path = "data/glove.6B.100d.txt"
bias = 3
glove_dict = {}
with open(glove_path, "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        glove_dict[word] = vector
# print(len(glove_dict))
# print(glove_dict.keys())
print(glove_dict['recent'].shape[-1])
mean_value = np.mean(list(glove_dict.values()))
vairance_value = np.var(list(glove_dict.values()))
left = mean_value - vairance_value * bias
right = mean_value + vairance_value * bias
for key in glove_dict.keys():
    temp_clip = np.clip(glove_dict[key], left, right)
    temp = (temp_clip - mean_value) / (bias * vairance_value)
    glove_dict[key] = temp + 1






100


In [1]:
from args import SNNArgs
from model.textcnn import TextCNN
from main import *
from snntorch import surrogate

arg = SNNArgs()
build_environment(arg)
model = TextCNN(arg, spike_grad=surrogate.fast_sigmoid).to(arg.device)

print(list(model._modules.values()))

/home/xujh/miniconda3/envs/snn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[ModuleList(
  (0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
  (1): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
  (2): Conv2d(1, 100, kernel_size=(5, 100), stride=(1, 1))
), ModuleList(
  (0): MaxPool2d(kernel_size=(23, 1), stride=(23, 1), padding=0, dilation=1, ceil_mode=False)
  (1): MaxPool2d(kernel_size=(22, 1), stride=(22, 1), padding=0, dilation=1, ceil_mode=False)
  (2): MaxPool2d(kernel_size=(21, 1), stride=(21, 1), padding=0, dilation=1, ceil_mode=False)
), Leaky(), Linear(in_features=300, out_features=2, bias=True), Leaky()]


In [8]:
# for c in model.convs_1:
#     print(c)
spikes = torch.tensor([0,1,1,1,1,1,0])
zeros = torch.eq(spikes, 0)
print(zeros)
print(torch.sum(zeros))
# print(zeros.shape[0].item())
print(7/2)
l = [0,2,3,4,5]
print(", ".join(["spike no.{}: dead ratio:{}".format(index, value) 
            for index, value in enumerate(l)]))

tensor([ True, False, False, False, False, False,  True])
tensor(2)
3.5
spike no.0: dead ratio0, spike no.1: dead ratio2, spike no.2: dead ratio3, spike no.3: dead ratio4, spike no.4: dead ratio5


In [4]:
import tensorflow as tf
sess = tf.compat.v1.Session()
x = tf.constant([[1,2,3,4], [4,3,2,5],[6,3,7,1], [8,6,4,1]])
_, indices = tf.nn.top_k(x, k=2, sorted=True)
print(indices)
top_1 = tf.slice(indices, [0,0], [-1,1])
print(top_1)
top_2 = tf.slice(indices, [0,0], [-1,2])
print(top_2)
sess.run()

tf.Tensor(
[[3 2]
 [3 0]
 [2 0]
 [0 1]], shape=(4, 2), dtype=int32)
tf.Tensor(
[[3]
 [3]
 [2]
 [0]], shape=(4, 1), dtype=int32)
tf.Tensor(
[[3 2]
 [3 0]
 [2 0]
 [0 1]], shape=(4, 2), dtype=int32)


TypeError: run() missing 1 required positional argument: 'fetches'

In [2]:
import copy
import torch

def permu(bit, dep, a, ret):
    if (dep == bit):
        ret.append(copy.deepcopy(a))
        return
    a[dep] = 1
    permu(bit, dep+1, a, ret)
    a[dep] = 0
    permu(bit, dep+1, a, ret)

def green(bit, dep, res, a, ret):
    if (res == 0):
        ret.append(copy.deepcopy(a))
        return
    for i in range(bit-res, dep-1, -1):
        a[i] = 0
        green(bit, i+1, res-1, a, ret)
        a[i] = 1

def preprocess(bit=8):
    a = [0 for _ in range(bit)]
    permu_a, greencode = [], []
    permu(bit, 0, a, permu_a)
    a = [1 for _ in range(bit)]
    for num0 in range(bit+1):
        green(bit, 0, num0, a, greencode)
    greencode = greencode[::-1]
    return permu_a, greencode

def encode(x, codebook, step_num=32, bit=8):
    bin_num = 1 << bit
    stride = 1. / bin_num
    bin_idx = min(int(x / stride), bin_num-1)
    spike = codebook[bin_idx]
    return int(step_num / bit) * spike

/home/xujh/miniconda3/envs/snn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
bit = 4
p_a, greencode = preprocess(bit=bit)
print(greencode)
print(p_a)
torch_green = torch.tensor(greencode, dtype=torch.float)
print(torch_green.shape)
bin_num = 1 << bit
stride = 1. / bin_num
temp = []
for i in range(10):
    temp += [i/10 for _ in range(10)]
x = torch.tensor(temp)
bin_idx = torch.clamp(torch.floor(x / stride), max=bin_num-1).int()
print(bin_idx)
y = torch.index_select(torch_green, 0, bin_idx)
print(y.shape)
temp_shape = [1 for _ in range(len(y.shape))]
temp_shape[-1] = 2
print(temp_shape)
print(y.repeat(temp_shape))



[[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 1], [0, 1, 0, 1], [0, 1, 1, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 1, 0, 0], [0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 1, 1]]
[[1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 0, 0], [1, 0, 1, 1], [1, 0, 1, 0], [1, 0, 0, 1], [1, 0, 0, 0], [0, 1, 1, 1], [0, 1, 1, 0], [0, 1, 0, 1], [0, 1, 0, 0], [0, 0, 1, 1], [0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, 0]]
torch.Size([16, 4])
tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 11, 11,
        11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        14, 14, 14, 14, 14, 14, 14, 14, 14, 14], dtype=torch.int32)
torch.Size([100, 4])
[1, 2]
tensor([[0., 0., 0., 0., 0., 0., 0